In [1]:
import os
import pandas as pd

In [2]:
os.chdir('./data')

In [17]:
car_detail = pd.read_excel('car_detail_all_raw.xlsx')
car_sold = pd.read_excel('monthly.xlsx')

In [18]:
car_sold.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15874 entries, 0 to 15873
Data columns (total 9 columns):
brand_cd      15874 non-null int64
brand_nm      15874 non-null object
seg_cd        15874 non-null object
seg_nm        15874 non-null object
model         15874 non-null int64
YM            15874 non-null int64
title         15874 non-null object
sold          15874 non-null int64
seg_detail    15874 non-null int64
dtypes: int64(5), object(4)
memory usage: 1.1+ MB


In [19]:
car_detail.model = car_detail.model.astype('str')
car_detail.drop(columns='lu_nm',inplace=True) # lu_nm 드랍

idx_to_drop = car_detail[car_detail.duplicated(subset=None,keep='first')].index # 중복 드랍
car_detail.drop(index=idx_to_drop,inplace=True)

car_detail.reset_index(drop=True,inplace=True)

In [20]:
car_detail.head()

,model,lu_cd,trim,trim_idx,price,specs,opt
0,2132,37482,VAN M/T,0,8710000,"{'치수': '3,595', '전폭(㎜)': '1,595', '전고(㎜)': '1,...","{'안전': '운전석, 동반석', '주행안전': 'ABS, 급제동 경보 시스템(ES..."
1,2132,37482,VAN A/T,1,9960000,"{'치수': '3,595', '전폭(㎜)': '1,595', '전고(㎜)': '1,...","{'안전': '운전석, 동반석', '주행안전': 'ABS, 급제동 경보 시스템(ES..."
2,2132,37482,VAN 고급형 M/T,2,9060000,"{'치수': '3,595', '전폭(㎜)': '1,595', '전고(㎜)': '1,...","{'안전': '운전석, 동반석', '주행안전': 'ABS, 급제동 경보 시스템(ES..."
3,2132,37482,VAN 고급형 A/T,3,10310000,"{'치수': '3,595', '전폭(㎜)': '1,595', '전고(㎜)': '1,...","{'안전': '운전석, 동반석', '주행안전': 'ABS, 급제동 경보 시스템(ES..."
4,2132,37482,스마트 M/T,4,9110000,"{'치수': '3,595', '전폭(㎜)': '1,595', '전고(㎜)': '1,...","{'안전': '운전석, 동반석, 사이드(앞), 커튼(앞뒤)', '주행안전': 'AB..."


In [21]:
car_detail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6680 entries, 0 to 6679
Data columns (total 7 columns):
model       6680 non-null object
lu_cd       6680 non-null int64
trim        6680 non-null object
trim_idx    6680 non-null int64
price       6680 non-null int64
specs       6680 non-null object
opt         6680 non-null object
dtypes: int64(3), object(4)
memory usage: 365.4+ KB


specs와 opt의 처리

In [22]:
car_detail.specs = pd.Series(map(eval, car_detail.specs)) # str인 제원과 옵션정보 dict으로
car_detail.opt = pd.Series(map(eval, car_detail.opt))

In [23]:
car_detail_spec = pd.DataFrame.from_dict(car_detail.specs.tolist()) # dict형태인 제원과 옵션정보의 인덱스를 보존한 채 df형태로
car_detail_opt = pd.DataFrame.from_dict(car_detail.opt.tolist())

In [24]:
car_detail2 = pd.concat((car_detail.drop(columns=['specs','opt']),car_detail_spec,car_detail_opt),axis=1) # axis 1로 concat -> raw데이터 완성

In [25]:
car_detail2.describe(include='object')

,model,trim,(구기준)연비(㎞/ℓ),1회충전 주행거리(㎞),4WD 시스템,CO₂ 배출 (g/km),CO₂배출(g/㎞),"가속( 초,0→100㎞/h)",고(㎜),고속연비 (km/ℓ),...,주행안전,차음 유리,크루즈컨트롤,통풍시트,트렁크,파워 아웃렛,파워시트,파워윈도우,편의,후방충격 저감 시스템
count,6680,6680,184,7,1159,3474,3116,67,9,3474,...,6155,3143,2615,392,3143,3143,632,6001,6155,3143
unique,502,2189,41,3,15,194,186,7,1,137,...,219,7,3,7,5,12,20,15,99,2
top,3500,프레스티지 A/T,14.1,44,풀타임,,156,9.6,"1,225",,...,"ABS, 전자식 트랙션 콘트롤 , 스테빌리티 컨트롤, 전자식 제동력 분배 시스템, BAS",,기본적용,앞좌석,,"시거잭, USB",,"앞좌석, 뒷좌석",,
freq,170,117,24,4,444,421,76,29,9,434,...,976,2155,1910,164,1756,1575,208,2645,1413,2735


In [26]:
car_detail2.describe()

,lu_cd,trim_idx,price
count,6680.000000,6680.000000,6.680000e+03
mean,41524.877395,2.570808,4.886977e+07
std,2334.758282,3.566545,4.715439e+07
min,33966.000000,0.000000,0.000000e+00
25%,40346.000000,0.000000,2.419000e+07
50%,42142.500000,1.000000,3.440000e+07
75%,43378.250000,3.000000,5.590000e+07
max,44156.000000,39.000000,7.600000e+08


In [27]:
car_detail2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6680 entries, 0 to 6679
Columns: 166 entries, model to 후방충격 저감 시스템
dtypes: int64(3), object(163)
memory usage: 8.5+ MB


In [56]:
car_detail2.to_excel('car_detail_all_final.xlsx',encoding='utf-8')

이후 전처리 추가적으로 필요